In [20]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

In [ ]:
import mysql.connector
from mysql.connector import Error

def connect_db():
    try:
        connection = mysql.connector.connect(
            host="*****",
            port=*****,
            user="*****",
            password="******",         # blank if your MySQL root password is empty
            database="*****"   # change this to YOUR db name
        )

        if connection.is_connected():
            print("Connected to MySQL!")
            return connection

    except Error as e:
        print("Error while connecting:", e)
        return None

In [3]:
connect_db()

Connected to MySQL!


In [93]:
def run_query(query, params=None):
    conn = connect_db()
    if conn is None:
        return

    cursor = conn.cursor(dictionary=True)

    try:
        cursor.execute(query, params or ())
        result = cursor.fetchall()
        return result

    except Error as e:
        print("Query Error:", e)

    finally:
        cursor.close()
        conn.close()


# Example usage

In [94]:
Data_base = {
  "database_name": "company_transactions",
  "description": "A transactional database containing customers, their financial accounts, merchants, and all associated transactions across multiple countries and currencies.",

  "tables": {
    "customers": {
      "description": "Stores customer demographic and account-level segmentation details.",
      "primary_key": "id",
      "columns": {
        "id": "INTEGER - unique identifier for each customer",
        "full_name": "VARCHAR(100) - customer's full name",
        "email": "VARCHAR(150) - unique email address",
        "country": "VARCHAR(50) - customer's country",
        "segment": "VARCHAR(30) - customer type: retail, smb, enterprise",
        "created_at": "TIMESTAMP - when the customer was added"
      },
      "example_row": {
        "id": 1,
        "full_name": "Amit Patel",
        "email": "amit.patel@example.com",
        "country": "USA",
        "segment": "retail",
        "created_at": "2023-01-10 09:15:00"
      }
    },

    "accounts": {
      "description": "Financial accounts belonging to customers. Each customer can have multiple accounts.",
      "primary_key": "id",
      "foreign_keys": {
        "customer_id": "customers.id"
      },
      "columns": {
        "id": "INTEGER - unique account identifier",
        "customer_id": "INTEGER - links account to customer",
        "account_type": "VARCHAR(30) - checking, savings, credit",
        "currency": "VARCHAR(10) - account currency",
        "status": "VARCHAR(20) - active, frozen, closed",
        "opened_at": "TIMESTAMP - account creation time"
      },
      "example_row": {
        "id": 101,
        "customer_id": 1,
        "account_type": "checking",
        "currency": "USD",
        "status": "active",
        "opened_at": "2023-01-10 09:20:00"
      }
    },

    "merchants": {
      "description": "Directory of merchants where transactions occur.",
      "primary_key": "id",
      "columns": {
        "id": "INTEGER - unique merchant ID",
        "name": "VARCHAR(120) - merchant name",
        "category": "VARCHAR(50) - grocery, travel, electronics, restaurant, etc.",
        "country": "VARCHAR(50) - country where merchant operates"
      },
      "example_row": {
        "id": 201,
        "name": "FreshMart Supermarket",
        "category": "grocery",
        "country": "USA"
      }
    },

    "transactions": {
      "description": "Financial transactions including purchases, refunds, fees, transfers, and chargebacks.",
      "primary_key": "id",
      "foreign_keys": {
        "account_id": "accounts.id",
        "merchant_id": "merchants.id (nullable)",
        "related_txn_id": "transactions.id (for refunds and chargebacks)"
      },
      "columns": {
        "id": "INTEGER - unique transaction ID",
        "account_id": "INTEGER - account used for the transaction",
        "merchant_id": "INTEGER - merchant involved (NULL for fees/transfers)",
        "txn_time": "TIMESTAMP - time of transaction",
        "amount": "DECIMAL(12,2) - positive or negative amount",
        "currency": "VARCHAR(10) - transaction currency",
        "txn_type": "VARCHAR(20) - purchase, refund, fee, transfer_in, transfer_out, chargeback",
        "channel": "VARCHAR(20) - pos, online, atm, manual",
        "status": "VARCHAR(20) - settled, pending, declined, reversed, chargeback",
        "related_txn_id": "INTEGER - reference to original txn for refunds/chargebacks"
      },
      "example_row": {
        "id": 1001,
        "account_id": 101,
        "merchant_id": 201,
        "txn_time": "2023-07-01 09:15:00",
        "amount": 82.45,
        "currency": "USD",
        "txn_type": "purchase",
        "channel": "pos",
        "status": "settled",
        "related_txn_id": "null"
      }
    }
  },

  "relationships": [
    {
      "type": "one_to_many",
      "from_table": "customers",
      "from_key": "id",
      "to_table": "accounts",
      "to_key": "customer_id",
      "description": "Each customer can have multiple accounts."
    },
    {
      "type": "one_to_many",
      "from_table": "accounts",
      "from_key": "id",
      "to_table": "transactions",
      "to_key": "account_id",
      "description": "Each account can have multiple transactions."
    },
    {
      "type": "many_to_one",
      "from_table": "transactions",
      "from_key": "merchant_id",
      "to_table": "merchants",
      "to_key": "id",
      "description": "Each transaction may involve one merchant."
    },
    {
      "type": "self_reference",
      "table": "transactions",
      "key": "related_txn_id",
      "description": "Refunds or chargebacks reference an original transaction."
    }
  ]
}


In [95]:
Prompt = f"""
You are an SQL expert. You know how to write complex SQL queries using JOINs, GROUP BY, window functions, CTEs, subqueries, and all major SQL features.

You have access to the following database schema:

{Data_base}

Your behavior rules:

1. GREETINGS:
   If the user says "hi", "hello", "hey", etc., reply normally with a greeting.
   DO NOT use JSON format for greetings.

2. DATABASE DESCRIPTION QUESTIONS:
   If the user asks general questions like:
   - "Tell me about the database"
   - "What tables exist?"
   - "What does this database contain?"
   Then provide a short explanation.
   DO NOT use JSON format for this.

3. SQL QUESTIONS:
   If the user asks something that requires data retrieval, return ONLY the following JSON:
   {{
     "sql": "<SQL_QUERY>",
     "is_db_question": true
   }}

4. NON-DATABASE QUESTIONS:
   If the question is NOT about the database and not a greeting, return JSON:
   {{
     "sql": null,
     "is_db_question": false
   }}

Example SQL Input:
User: "I want all the top merchants by revenue."

Example SQL Output:
{{
  "sql": "SELECT m.name AS merchant, m.category, SUM(t.amount) AS total_revenue FROM merchants m JOIN transactions t ON m.id = t.merchant_id WHERE t.status = 'settled' GROUP BY m.name, m.category ORDER BY total_revenue DESC;",
  "is_db_question": true
}}

Follow this decision logic exactly.
"""


In [96]:
Question = "Average transaction amount by country"
data = [{'role': 'system', 'content': Prompt}] + [{'role': 'user', 'content': Question}]


In [97]:
openai = OpenAI()

response = openai.chat.completions.create(

        model="gpt-4o-mini",
        messages=data,

)

In [91]:
json.loads(response.choices[0].message.content)

{'sql': "SELECT c.country, AVG(t.amount) AS average_transaction_amount FROM customers c JOIN accounts a ON c.id = a.customer_id JOIN transactions t ON a.id = t.account_id WHERE t.status = 'settled' GROUP BY c.country;",
 'is_db_question': True}

In [98]:
output  =   json.loads(response.choices[0].message.content)

run_query(output['sql'])

Connected to MySQL!


[{'country': 'USA', 'average_transaction_amount': Decimal('84.297778')},
 {'country': 'India', 'average_transaction_amount': Decimal('540.000000')},
 {'country': 'Canada', 'average_transaction_amount': Decimal('59.000000')},
 {'country': 'Mexico', 'average_transaction_amount': Decimal('3500.000000')},
 {'country': 'UK', 'average_transaction_amount': Decimal('30.740000')},
 {'country': 'Brazil', 'average_transaction_amount': Decimal('67.500000')},
 {'country': 'France', 'average_transaction_amount': Decimal('42.300000')}]

In [117]:
def chat(messgae, history):
    full_message = [{'role': 'system', 'content': Prompt}] + history + [{'role': 'user', 'content': messgae}]
    openai = OpenAI()

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=full_message,
    )

    try:
        output = json.loads(response.choices[0].message.content)
    except json.JSONDecodeError:
        # This is likely a greeting or general explanation
        return {'role': 'system', 'content': response.choices[0].message.content}

    if output['is_db_question']:
# 
        answer = run_query(output['sql'])

        # full_message.extend(answer)
        answer_text = json.dumps(answer, default=str)

        # Append result as a message
        new_data = [{'role':'system','content':'Beautfy the answer for the user'}]+[{'role': 'system', 'content': f"SQL result: {answer_text}"}]

        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=new_data,
        )
        # print(response.choices[0].message.content)

        return {'role': 'system', 'content': response.choices[0].message.content}

    return {'role': 'system', 'content': response.choices[0].message.content}


In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7874
* To create a public link, set `share=True` in `launch()`.


Connected to MySQL!
Here's a beautifully organized summary of transaction counts by country:

- **United States**: 10 transactions
- **India**: 5 transactions
- **Canada**: 2 transactions
- **Mexico**: 2 transactions
- **United Kingdom**: 2 transactions
- **Brazil**: 2 transactions
- **France**: 2 transactions

This showcases a diverse range of activity across these nations!
